# Tablesaw 

[Tablesaw](https://jtablesaw.wordpress.com/) is easy to add to the BeakerX Groovy kernel.
Tablesaw provides the ability to easily transform, summarize, and filter data, as well as computing descriptive statistics and fundamental machine learning algorithms.

This notebook has some basic demos of how to use Tablesaw, including visualizing the results.  This notebook uses the Beaker interactive visualizaiton libraries, but Tablesaw's APIs also work.  The notebook covers basic table manipulation, k-means clustering, and linear regression.

Download the Tablesaw jar and put it in the same directory as this notebook is in.
(http://central.maven.org/maven2/com/github/lwhite1/tablesaw/0.7.6.4/tablesaw-0.7.6.4.jar)

In [ ]:
%classpath add jar tablesaw-0.7.6.4.jar
%import com.github.lwhite1.tablesaw.api.*
%import com.github.lwhite1.tablesaw.api.ml.clustering.*
%import com.github.lwhite1.tablesaw.api.ml.regression.*
%import com.github.lwhite1.tablesaw.columns.*    
%import com.github.lwhite1.tablesaw.reducing.*    

In [ ]:
tornadoes = Table.createFromCsv("Tablesaw/tornadoes_2014.csv")
//print dataset structure    
tornadoes.structure().print()

In [ ]:
//get header names
tornadoes.columnNames()

In [ ]:
//displays the row and column counts
tornadoes.shape()

In [ ]:
//displays the first n rows
tornadoes.first(10).print()

In [ ]:
import static com.github.lwhite1.tablesaw.api.QueryHelper.column
tornadoes.structure().selectWhere(column("Column Type").isEqualTo("FLOAT")).print()

In [ ]:
//summarize the data in each column
tornadoes.summary()

In [ ]:
//Mapping operations
def month = tornadoes.dateColumn("Date").month()
tornadoes.addColumn(month);
tornadoes.columnNames()

In [ ]:
//Sorting by column
tornadoes.sortOn("-Fatalities").print()

In [ ]:
//Descriptive statistics
tornadoes.column("Fatalities").summary().print();

In [ ]:
//Performing totals and sub-totals
def injuriesByScale = tornadoes.median("Injuries").by("Scale")
injuriesByScale.setName("Median injuries by Tornado Scale")
injuriesByScale.print()

In [ ]:
//Cross Tabs
CrossTab.xCount(tornadoes, tornadoes.categoryColumn("State"), tornadoes.shortColumn("Scale")).print()

## K-means clustering

K-means is the most common form of “centroid” clustering. Unlike classification, clustering is an unsupervised learning method. The categories are not predetermined. Instead, the goal is to search for natural groupings in the dataset, such that the members of each group are similar to each other and different from the members of the other groups. The K represents the number of groups to find.

We’ll use a well known Scotch Whiskey dataset, which is used to cluster whiskeys according to their taste based on data collected from tasting notes. As always, we start by loading data and printing its structure.

More description is available at https://jtablesaw.wordpress.com/2016/08/08/k-means-clustering-in-java/

In [ ]:
t = Table.createFromCsv("Tablesaw/whiskey.csv")
t.structure().print();

In [ ]:
model = new Kmeans(
    5,
    t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
    t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
);

//print claster formation
model.clustered(t.column("Distillery")).print();

In [ ]:
//print centroids for each claster
model.labeledCentroids().print();

In [ ]:
//gets the distortion for our model
model.distortion()

In [ ]:
def n = t.rowCount();
def kValues = new double[n - 2];
def distortions = new double[n - 2];

for (int k = 2; k < n; k++) {
  kValues[k - 2] = k;
  def kmeans = new Kmeans(k,
      t.nCol(2), t.nCol(3), t.nCol(4), t.nCol(5), t.nCol(6), t.nCol(7),
      t.nCol(8), t.nCol(9), t.nCol(10), t.nCol(11), t.nCol(12), t.nCol(13)
  );
  distortions[k - 2] = kmeans.distortion();
}
def linearYPlot = new Plot(title: "K-means clustering demo", xLabel:"K", yLabel: "distortion")
linearYPlot << new Line(x: kValues, y: distortions)

## Play (Money)ball with Linear Regression

In baseball, you make the playoffs by winning more games than your rivals. The number of games the rivals win is out of your control so the A’s looked instead at how many wins it took historically to make the playoffs. They decided that 95 wins would give them a strong chance.  Here’s how we might check that assumption in Tablesaw.

More description is available at https://jtablesaw.wordpress.com/2016/07/31/play-moneyball-data-science-in-tablesaw/

In [ ]:
import static com.github.lwhite1.tablesaw.api.QueryHelper.column

baseball = Table.createFromCsv("Tablesaw/baseball.csv");

// filter to the data available at the start of the 2002 season
moneyball = baseball.selectWhere(column("year").isLessThan(2002));
wins = moneyball.nCol("W");
year = moneyball.nCol("Year");
playoffs = moneyball.column("Playoffs");
runDifference = moneyball.shortColumn("RS").subtract(moneyball.shortColumn("RA"));
moneyball.addColumn(runDifference);
runDifference.setName("RD");

def Plot = new Plot(title: "RD x Wins", xLabel:"RD", yLabel: "W")
Plot << new Points(x: moneyball.numericColumn("RD").toDoubleArray(), y: moneyball.numericColumn("W").toDoubleArray())

In [ ]:
winsModel = LeastSquares.train(wins, runDifference);

In [ ]:
def runDiff = new double[1];
runDiff[0] = 135;
def expectedWins = winsModel.predict(runDiff);
runsScored2 = LeastSquares.train(moneyball.nCol("RS"), moneyball.nCol("OBP"), moneyball.nCol("SLG"));

In [ ]:
new Histogram(xLabel:"X",
              yLabel:"Proportion",
              data: Arrays.asList(runsScored2.residuals()), 
              binCount: 25);